# Tecnologias de Processamento de dados - 2019/2020

## Phase II - Group 12


|   Student      | Student ID |  Contribution in hours |
|----------------|------------|----------------|
| Beatriz Lima   |    49377   |   |
| David Almeida  |    54120   |   |
|João Castanheira|    55052   |   |
| Pedro Cotovio  |    55053   |   |





## 0. Get the data

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import norm

The main datasets used in this warehouse are:

- http://insideairbnb.com/get-the-data.html for Lisbon, Portugal. - listings.csv
- https://dadosabertos.turismodeportugal.pt/datasets/alojamento-local) - Alojamento_Local.csv

In [5]:
listings_file_path = '../data/airbnb/listings.csv'
al_file_path = '../data/Alojamento_Local.csv'
df_al = pd.read_csv(al_file_path)
df_listings = pd.read_csv(listings_file_path)

/Users/air/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Merge _df_listings_ with _alojamento_local.csv_

In order to enrich the main dataset, we can cross it with the dataset from Registo Nacional de Alojamento Local (RNAL) to obtain further information regarding each listing's property, as well as refine already available data, particularly in the case of location data.

In [7]:
def intTryParse(value):
    """Tries to parse string to an integer"""
    try:
        a = int(value)
        return True
    except ValueError:
        return False

In [9]:
# get only listings where 
df_listings_with_license = df_listings[(~df_listings['license'].isnull()) #'license' is not null
                                        & (df_listings['license'] != 'Exempt')] # && != 'Exempt'

# string replace
df_listings_with_license['NrRNAL'] = [s.replace('/AL','').replace('.','') # remove '/AL' and '.' from code
                                      for s in df_listings_with_license['license']]

# get only records where license nr can be converted to int 
df_listings_with_license = df_listings_with_license[[intTryParse(s) # if code can be converted to int
                                                     for s in df_listings_with_license['NrRNAL']]] # keep it

# convert NrRNAL to int before merge the two dataframes
df_listings_with_license['NrRNAL'] = df_listings_with_license['NrRNAL'].astype(np.int64) # convert code to int

# inner join two dataframes
df_result = pd.merge(df_listings_with_license, df_al, how='inner', on='NrRNAL')

/Users/air/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Save the intersection of the two files to disk:

In [11]:
listings__al_file_path = '../data/listings_al.csv'
df_result.to_csv(listings__al_file_path,index=False)
print('Dataset size: {}'.format(len(df_result)))

Dataset size: 17168


## 1. Dimensions and facts tables of the data warehouse

+ Define and model them in SQL
+ Identify hierarchies and fact granularity
+ Create the dimensions and facts tables in the DBMS (postgreSQL)

### General schema

![Star schema](../schema.png)

## 2. Define an ETL workflow

+ Identify all data sources for all dimensions. Add URL links to all data that should be available. If not public data, point to dropbox files, Google drive, or whatever
+ For each dimension show the code used for modeling, filtering and inserting data
+ Describe the process for inserting facts data

The ETL workflow is defined in separate notebooks for each dimension:

 - ETL_Property.ipynb
 - ETL_Host.ipynb
 - ETL_Review.ipynb
 - ETL_Date.ipynb
 - ETL_Location.ipynb

## 3. Critical assessment of the work
+ Describe potential issues with the ETL procedure used
+ Compare your schema to the one previously defined in phase I
+ Discuss the issues for updating the data warehouse with novel data